In [4]:
import pandas as pd
import psycopg2

# Connecting to the PostgreSQL Database
conn = psycopg2.connect(
    dbname = "selfproject1",
    user = "postgres",
    password = "!SuperShia3",
    host = "localhost",
    port = "4477"
)
cur = conn.cursor()

In [5]:
# Loading in the Excel Workbook
file = "/Users/alijazibrizvi/Documents/Data Analytics/Self-Project | Social Science & Demography/ProjectData.xlsm"

In [16]:
# Using Pandas to Read all Sheets in the Excel Workbook
xls = pd.ExcelFile(file)

# Looping through each Sheet
for sheet_name in xls.sheet_names:
    try:
        ## Reading each Sheet into a Pandas DataFrame
        df = pd.read_excel(file, sheet_name=sheet_name)

        ## Cleaning the Column Names: Removing Special Characters and Spaces
        df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True).str.lower()

        ## Ensure column names that start with a number are prefixed with 'col_'
        df.columns = [f'col_{col}' if col[0].isdigit() else col for col in df.columns]

        ## Cleaning the Sheet Name (Table Name):
        cleaned_sheet_name = sheet_name.lower().replace('-', '_').replace('&', 'and').replace(' ', '_')

        # Ensure table names that start with a number are prepended with 'tbl_'
        if cleaned_sheet_name[0].isdigit():
            cleaned_sheet_name = 'tbl_' + cleaned_sheet_name

        ## Creating a Table Dynamically, based on the Cleaned Sheet Name
        create_table_query = f'''
        CREATE TABLE IF NOT EXISTS {cleaned_sheet_name} (
        {', '.join([f"{col} VARCHAR(255)" for col in df.columns])}
        );
        '''

        # Execute the create table query
        cur.execute(create_table_query)

        # Insert data into the table
        for index, row in df.iterrows():
            insert_query = f'''
            INSERT INTO {cleaned_sheet_name} ({', '.join(df.columns)})
            VALUES ({', '.join(['%s'] * len(df.columns))});
            '''
            cur.execute(insert_query, tuple(row))
        
        # Commit after each sheet's data is inserted
        conn.commit()
    
    except Exception as e:
        print(f"Error creating table for {sheet_name}: {e}")
        conn.rollback()  # Rollback if there is an error

In [17]:
# Close the Cursor and the Connection
cur.close()
conn.close()